<a href="https://colab.research.google.com/github/dung18520632/CS114.K21/blob/master/VGG_16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **CÀI ĐẶT CÁC THƯ VIỆN**

In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import tensorflow as tf
import cv2
from PIL import Image
import os

# **CONNECTED MY KAGGLE**

In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"hoangvandung","key":"536b985b25a4e735caa0dff6dcb92a3b"}'}

In [3]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# **TẢI XUỐNG BỘ DATASET BIỂN BÁO**

In [4]:
!kaggle datasets download -d meowmeowmeowmeowmeow/gtsrb-german-traffic-sign --force

 98% 602M/612M [00:15<00:00, 32.4MB/s]
100% 612M/612M [00:15<00:00, 40.7MB/s]


# **GIẢI NÉN DATASET**

In [ ]:
!unzip gtsrb-german-traffic-sign.zip

# **TẢI DỮ LIỆU IMAGE VÀ LABEL**

In [6]:
data=[]
labels=[]

for i in range(43) :
    path = "/content/Train/{0}/".format(i)
    Class=os.listdir(path)#Trả về các thư mục con từ đường dẫn đã cho
    print(path)
    for a in Class:
        try:
            image=cv2.imread(path+a)
            image_from_array = Image.fromarray(image, 'RGB')#Chuyển các bức ảnh thành xám
            size_image = image_from_array.resize((64, 64))# Các bức ảnh có các size khác nhau, ta thay đổi size của chúng về chung (64,64)
            data.append(np.array(size_image))#Thêm ảnh vào data dưới dạng ma trận
            labels.append(i)#Đánh số labels
        except AttributeError:
            print(" ")

/content/Train/0/
/content/Train/1/
/content/Train/2/
/content/Train/3/
/content/Train/4/
/content/Train/5/
/content/Train/6/
/content/Train/7/
/content/Train/8/
/content/Train/9/
/content/Train/10/
/content/Train/11/
/content/Train/12/
/content/Train/13/
/content/Train/14/
/content/Train/15/
/content/Train/16/
/content/Train/17/
/content/Train/18/
/content/Train/19/
/content/Train/20/
/content/Train/21/
/content/Train/22/
/content/Train/23/
/content/Train/24/
/content/Train/25/
/content/Train/26/
/content/Train/27/
/content/Train/28/
/content/Train/29/
/content/Train/30/
/content/Train/31/
/content/Train/32/
/content/Train/33/
/content/Train/34/
/content/Train/35/
/content/Train/36/
/content/Train/37/
/content/Train/38/
/content/Train/39/
/content/Train/40/
/content/Train/41/
/content/Train/42/


# **KÍCH THƯỚC DATA VÀ LABELS**

In [7]:
print(len(data))
print(len(labels))

39209
39209


# **CHUYỂN DỮ LIỆU THÀNH MA TRẬN** 

In [8]:
Cells=np.array(data)
labels=np.array(labels)
#Lưu ma trận 
np.save("Cells",Cells)
np.save("labels",labels)

In [9]:
#Load ma trận
Cells=np.load("Cells.npy")
labels=np.load("labels.npy")

# **TẠO RA BỘ DỮ LIỆU NGẪU NHIÊN**

In [18]:
s=np.arange(Cells.shape[0])
np.random.seed(43)
np.random.shuffle(s)
Cells=Cells[s]
labels=labels[s]

# **CHIA DỮ LIỆU THÀNH PHẦN TRAIN VÀ PHẦN TEST**

In [19]:
(trainX,testX)=Cells[(int)(0.2*len(labels)):],Cells[:(int)(0.2*len(labels))]
trainX = trainX.astype('float32')/255 #
testX = testX.astype('float32')/255
(trainY,testY)=labels[(int)(0.2*len(labels)):],labels[:(int)(0.2*len(labels))]
#Using one hote encoding for the train and validation labels
from keras.utils import to_categorical
trainY = to_categorical(trainY, 43)
testY = to_categorical(testY, 43)
print(len(trainX))
print(len(trainY))
print(len(testX))
print(len(testY))


31368
31368
7841
7841


In [25]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras import applications
from tensorflow.keras import optimizers
model = VGG16(weights="imagenet", include_top=False,
	input_tensor=Input(trainX.shape[1:]))
x=Sequential()
x.add(Flatten(input_shape=model.output_shape[1:]))
x.add(Dense(256, activation='relu'))
x.add(Dense(43, activation='sigmoid'))
model = Model(inputs=model.input, outputs=x(model.output))
opt = Adam(lr=1e-4)
model.compile(loss="binary_crossentropy", optimizer=opt,
	metrics=["accuracy"])

model.summary()


Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 64, 64, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 64, 64, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 64, 64, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 32, 32, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 32, 32, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 32, 32, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 16, 16, 128)       0   

In [ ]:
print("[INFO] training ...")
model.fit(trainX, trainY, validation_data=(testX, testY),
  batch_size=64, epochs=10, verbose=1)


[INFO] training ...
Epoch 1/10
491/491 [==============================] - 93s 189ms/step - loss: 0.0280 - accuracy: 0.8236 - val_loss: 0.0022 - val_accuracy: 0.9860
Epoch 2/10
491/491 [==============================] - 93s 188ms/step - loss: 0.0016 - accuracy: 0.9920 - val_loss: 5.9654e-04 - val_accuracy: 0.9972
Epoch 3/10
491/491 [==============================] - 93s 189ms/step - loss: 0.0010 - accuracy: 0.9946 - val_loss: 4.4257e-04 - val_accuracy: 0.9971
Epoch 4/10
491/491 [==============================] - 93s 189ms/step - loss: 6.2069e-04 - accuracy: 0.9968 - val_loss: 4.0289e-04 - val_accuracy: 0.9985
Epoch 5/10
491/491 [==============================] - 93s 189ms/step - loss: 3.1445e-04 - accuracy: 0.9985 - val_loss: 3.3952e-04 - val_accuracy: 0.9986
Epoch 6/10
491/491 [==============================] - 93s 189ms/step - loss: 7.7246e-04 - accuracy: 0.9962 - val_loss: 4.8958e-04 - val_accuracy: 0.9973
Epoch 7/10
491/491 [==============================] - 93s 189ms/step - loss: 3

In [ ]:
from tensorflow.keras.models import load_model

print("[INFO] save model...")
model.save('/content/vgg16.model')

[INFO] save model...
INFO:tensorflow:Assets written to: /content/vgg16.model/assets


In [ ]:
from tensorflow.keras.models import load_model

print("[INFO] loading model...")
model = load_model('/content/vgg16.model')

[INFO] loading model...


In [ ]:
#Predicting with the test data
import pandas as pd
y_test=pd.read_csv("/content/Test.csv")
y_test.head()
labels=y_test['Path'].to_numpy() 
y_test=y_test['ClassId'].values

data=[]

for f in labels:
    image_test=cv2.imread('/content/test/'+f.replace('Test/', ''))
    image_from_array_test = Image.fromarray(image_test, 'RGB')
    size_image = image_from_array_test.resize((64, 64))
    data.append(np.array(size_image))

X_test=np.array(data)
X_test = X_test.astype('float32')/255 
predictions = model.predict(X_test ,batch_size=64)
pred = predictions.argmax(axis=1)

In [ ]:
from sklearn.metrics import accuracy_score

print(accuracy_score(y_test, pred))

0.9719714964370546
